<a href="https://colab.research.google.com/github/LukeMattingly/cartoonify-yourself/blob/master/Cartoonify_Yourself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers
!pip install opencv-contrib-python
!pip install diffusers transformers git+https://github.com/huggingface/accelerate.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-tcuz2mg4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-tcuz2mg4
  Resolved https://github.com/huggingface/accelerate.git to commit 2106e87d585ae9a245c895c568fffeaa519dfb9a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.0 MB/s eta 0:00:00
   

In [ ]:
import cv2
from PIL import Image
import numpy as np
from diffusers.utils import load_image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch

In [ ]:
# load image
image = load_image(
    "https://media.licdn.com/dms/image/C5603AQHKPqT68GrPkA/profile-displayphoto-shrink_400_400/0/1636174202437?e=1686787200&v=beta&t=9mePf1XfxLJTfTibQuK-HZVDsUGd7jwXNOD6SqLifSA"
)

In [ ]:
image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

In [ ]:
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)


pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# this command loads the individual model components on GPU on-demand.
pipe.enable_model_cpu_offload()

In [ ]:
generator = torch.manual_seed(0)

out_image = pipe(
    "cartoon character", num_inference_steps=20, generator=generator, image=canny_image
).images[0]

out_image

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/utils/_contextlib.py:115 in decorate_context        │
│                                                                                                  │
│   112 │   @functools.wraps(func)                                                                 │
│   113 │   def decorate_context(*args, **kwargs):                                                 │
│   114 │   │   with ctx_factory():                                                                │
│ ❱ 115 │   │   │   return func(*args, **kwargs)                                                   │
│   116 │                                                                                          │
│   117 │   return decorate_context                                                                │
│   118                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diff │
│ usion_controlnet.py:865 in __call__                                                              │
│                                                                                                  │
│    862 │   │   │   controlnet_conditioning_scale = [controlnet_conditioning_scale] * len(self.c  │
│    863 │   │                                                                                     │
│    864 │   │   # 3. Encode input prompt                                                          │
│ ❱  865 │   │   prompt_embeds = self._encode_prompt(                                              │
│    866 │   │   │   prompt,                                                                       │
│    867 │   │   │   device,                                                                       │
│    868 │   │   │   num_images_per_prompt,                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diff │
│ usion_controlnet.py:389 in _encode_prompt                                                        │
│                                                                                                  │
│    386 │   │   │   │   attention_mask = None                                                     │
│    387 │   │   │                                                                                 │
│    388 │   │   │   prompt_embeds = self.text_encoder(                                            │
│ ❱  389 │   │   │   │   text_input_ids.to(device),                                                │
│    390 │   │   │   │   attention_mask=attention_mask,                                            │
│    391 │   │   │   )                                                                             │
│    392 │   │   │   prompt_embeds = prompt_embeds[0]                                              │
│                                                                                                  │
│ /usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:247 in _lazy_init                  │
│                                                                                                  │
│    244 │   │   # are found or any other error occurs                                             │
│    245 │   │   if 'CUDA_MODULE_LOADING' not in os.environ:                                       │
│    246 │   │   │   os.environ['CUDA_MODULE_LOADING'] = 'LAZ